In [1]:
import pandas as pd
import numpy as np
import nltk


from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer
import re
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/ismaildibirov/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [2]:
data = pd.read_csv("../data/capstone.messages.csv")
data.head()

,_id,channel,text,date
0,6579a3c0316b8a2a5b0c0827,cryptowhalesreal,I hope eveyone could get this fabulous profit ...,2023-12-12T09:40:12.000Z
1,6579a3c0316b8a2a5b0c0828,cryptowhalesreal,🎁 #FREE_SIGNAL \n\n🟣 Future Signal 🟣\n📈 BUY\n❇...,2023-12-11T12:39:33.000Z
2,6579a3c0316b8a2a5b0c0829,cryptowhalesreal,Another 40% profit after the entry for a buy s...,2023-12-10T13:24:29.000Z
3,6579a3c0316b8a2a5b0c082a,cryptowhalesreal,🟣 Future Signal 🟣\n📈 BUY\n❇️ #SOL - USDT\n🏢 Ex...,2023-12-10T13:24:20.000Z
4,6579a3c0316b8a2a5b0c082b,cryptowhalesreal,This is the correct way of starting the weeken...,2023-12-09T07:54:00.000Z


In [4]:
# Correcting the function to effectively remove all hanging 'n's
def clean_text_final_corrected(text):
    # Removing line breaks and replacing with space
    text = text.replace('\n', ' ')
    # Removing any standalone 'n' characters that might have been left from '\n'
    text = re.sub(r'\b[n]\b', ' ', text)
    # Remaining cleaning steps
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\@\w+|\#', '', text)
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
    text = re.sub(r'\s+', ' ', text)  # Removing extra space
    return text.lower()

# Reapplying the corrected text cleaning function
data['cleaned_text'] = data['text'].apply(clean_text_final_corrected)

# Displaying the first few rows of the updated cleaned data
data[['text', 'cleaned_text']].head(10)


,text,cleaned_text
0,I hope eveyone could get this fabulous profit ...,i hope eveyone could get this fabulous profit ...
1,🎁 #FREE_SIGNAL \n\n🟣 Future Signal 🟣\n📈 BUY\n❇...,free signal future signal buy tia usdt exchan...
2,Another 40% profit after the entry for a buy s...,another 40 profit after the entry for a buy si...
3,🟣 Future Signal 🟣\n📈 BUY\n❇️ #SOL - USDT\n🏢 Ex...,future signal buy sol usdt exchange all avail...
4,This is the correct way of starting the weeken...,this is the correct way of starting the weeken...
5,🟣 Future Signal 🟣\n📈 BUY\n❇️ #SXP - USDT\n🏢 Ex...,future signal buy sxp usdt exchange all avail...
6,Semms #YGG doen't have enough strength. We jus...,semms ygg doen t have enough strength we just ...
7,The price had a bearish shadow; don't worry 😮‍...,the price had a bearish shadow don t worry nti...
8,🎁 #FREE_SIGNAL \n\n🟣 Future Signal 🟣\n📈 BUY\n❇...,free signal future signal buy ygg usdt exchan...
9,We keep the Monday Blues away with profitable ...,we keep the monday blues away with profitable ...


In [5]:
# Manual tokenization function
def manual_tokenize(text):
    # Simple tokenization by splitting on spaces (this is less sophisticated than NLTK's word_tokenize)
    return text.split()

# Applying manual tokenization to the cleaned text
data['manual_tokens'] = data['cleaned_text'].apply(manual_tokenize)

# Displaying the first few rows with manual tokens
data[['cleaned_text', 'manual_tokens']].head()


,cleaned_text,manual_tokens
0,i hope eveyone could get this fabulous profit ...,"[i, hope, eveyone, could, get, this, fabulous,..."
1,free signal future signal buy tia usdt exchan...,"[free, signal, future, signal, buy, tia, usdt,..."
2,another 40 profit after the entry for a buy si...,"[another, 40, profit, after, the, entry, for, ..."
3,future signal buy sol usdt exchange all avail...,"[future, signal, buy, sol, usdt, exchange, all..."
4,this is the correct way of starting the weeken...,"[this, is, the, correct, way, of, starting, th..."


In [6]:
# Manually defining a basic list of English stopwords
basic_stopwords = set([
    "i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", 
    "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", 
    "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", 
    "theirs", "themselves", "what", "which", "who", "whom", "this", "that", 
    "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", 
    "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", 
    "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", 
    "at", "by", "for", "with", "about", "against", "between", "into", "through", 
    "during", "before", "after", "above", "below", "to", "from", "up", "down", 
    "in", "out", "on", "off", "over", "under", "again", "further", "then", 
    "once", "here", "there", "when", "where", "why", "how", "all", "any", 
    "both", "each", "few", "more", "most", "other", "some", "such", "no", 
    "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", 
    "can", "will", "just", "don", "should", "now"
])

# Function to remove stop words from tokens using the manually defined list
def remove_manual_stop_words(tokens):
    return [word for word in tokens if word not in basic_stopwords]

# Applying the stop word removal function using the manual list
data['tokens_no_stopwords_manual'] = data['manual_tokens'].apply(remove_manual_stop_words)

# Displaying the first few rows with tokens without stop words
data[['manual_tokens', 'tokens_no_stopwords_manual']].head()


,manual_tokens,tokens_no_stopwords_manual
0,"[i, hope, eveyone, could, get, this, fabulous,...","[hope, eveyone, could, get, fabulous, profit, ..."
1,"[free, signal, future, signal, buy, tia, usdt,...","[free, signal, future, signal, buy, tia, usdt,..."
2,"[another, 40, profit, after, the, entry, for, ...","[another, 40, profit, entry, buy, signal, cryp..."
3,"[future, signal, buy, sol, usdt, exchange, all...","[future, signal, buy, sol, usdt, exchange, ava..."
4,"[this, is, the, correct, way, of, starting, th...","[correct, way, starting, weekend, ntouched, ta..."


In [7]:
# Simplified lemmatization function (without using WordNet)
# This function will perform a basic form of lemmatization based on simple heuristics

def simplified_lemmatize(word):
    # Basic rules for lemmatization (this is a very rudimentary approach)
    if word.endswith('ing'):
        return word[:-3]
    if word.endswith('ed'):
        return word[:-2]
    if word.endswith('s'):
        return word[:-1]
    return word

def apply_simplified_lemmatization(tokens):
    return [simplified_lemmatize(word) for word in tokens]

# Applying simplified lemmatization to the tokens
data['simplified_lemmatized_tokens'] = data['tokens_no_stopwords_manual'].apply(apply_simplified_lemmatization)

# Displaying the first few rows with simplified lemmatized tokens
data[['tokens_no_stopwords_manual', 'simplified_lemmatized_tokens']].head()


,tokens_no_stopwords_manual,simplified_lemmatized_tokens
0,"[hope, eveyone, could, get, fabulous, profit, ...","[hope, eveyone, could, get, fabulou, profit, f..."
1,"[free, signal, future, signal, buy, tia, usdt,...","[free, signal, future, signal, buy, tia, usdt,..."
2,"[another, 40, profit, entry, buy, signal, cryp...","[another, 40, profit, entry, buy, signal, cryp..."
3,"[future, signal, buy, sol, usdt, exchange, ava...","[future, signal, buy, sol, usdt, exchange, ava..."
4,"[correct, way, starting, weekend, ntouched, ta...","[correct, way, start, weekend, ntouch, target,..."


In [8]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Creating a CountVectorizer object for BoW
bow_vectorizer = CountVectorizer(max_features=1000)  # Limiting to top 1000 features for simplicity
bow_features = bow_vectorizer.fit_transform(data['cleaned_text']).toarray()

# Creating a TfidfVectorizer object for TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=1000)  # Limiting to top 1000 features
tfidf_features = tfidf_vectorizer.fit_transform(data['cleaned_text']).toarray()

# Converting the features into DataFrames for better readability
bow_df = pd.DataFrame(bow_features, columns=bow_vectorizer.get_feature_names_out())
tfidf_df = pd.DataFrame(tfidf_features, columns=tfidf_vectorizer.get_feature_names_out())

# Displaying the first few rows of each DataFrame
bow_df.head(), tfidf_df.head()


(   00  000  01  06  0x  10  100  1000  11  12  ...  wrote  x50  year  years  \
 0   0    0   0   0   0   0    0     0   0   0  ...      0    0     0      0   
 1   0    0   0   1   0   6    0     0   1   1  ...      0    0     0      0   
 2   0    0   0   0   0   0    0     0   0   0  ...      0    0     0      0   
 3   0    0   0   0   0   1    0     0   1   0  ...      0    0     0      0   
 4   0    0   0   0   0   0    0     0   0   0  ...      0    0     0      0   
 
    yet  you  your  zero  zhao  zone  
 0    0    0     0     0     0     0  
 1    0    0     0     0     0     0  
 2    0    0     0     0     0     0  
 3    0    0     0     0     0     0  
 4    0    0     0     0     0     0  
 
 [5 rows x 1000 columns],
     00  000   01        06   0x        10  100  1000        11        12  ...  \
 0  0.0  0.0  0.0  0.000000  0.0  0.000000  0.0   0.0  0.000000  0.000000  ...   
 1  0.0  0.0  0.0  0.147065  0.0  0.475123  0.0   0.0  0.117719  0.104366  ...   
 2  0.0  0

In [10]:

# Initializing VADER's SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

# Function to get the compound sentiment score
def get_sentiment(text):
    return sia.polarity_scores(text)['compound']

# Applying the sentiment analysis to the cleaned text
data['sentiment_score'] = data['cleaned_text'].apply(get_sentiment)

# Categorizing sentiment into positive, neutral, or negative based on the compound score
data['sentiment_category'] = data['sentiment_score'].apply(lambda score: 'positive' if score > 0.05 else ('negative' if score < -0.05 else 'neutral'))

# Displaying the first few rows with sentiment scores and categories
data[['cleaned_text', 'sentiment_score', 'sentiment_category']].head(10)


,cleaned_text,sentiment_score,sentiment_category
0,i hope eveyone could get this fabulous profit ...,0.9468,positive
1,free signal future signal buy tia usdt exchan...,0.4767,positive
2,another 40 profit after the entry for a buy si...,0.7351,positive
3,future signal buy sol usdt exchange all avail...,-0.5423,negative
4,this is the correct way of starting the weeken...,0.7351,positive
5,future signal buy sxp usdt exchange all avail...,-0.5423,negative
6,semms ygg doen t have enough strength we just ...,0.9118,positive
7,the price had a bearish shadow don t worry nti...,-0.4404,negative
8,free signal future signal buy ygg usdt exchan...,-0.0516,negative
9,we keep the monday blues away with profitable ...,0.7351,positive
